# KNN算法

![KNN算法](../images/KNN/KNN算法.jpg)

![KNN图解](../images/KNN/KNN图解.jpg)

![KDTree](../images/KNN/KDTree构造1.jpg)

![KDTree](../images/KNN/KDTree构造2.jpg)

![KDTree](../images/KNN/KDTree搜索1.jpg)

![KDTree](../images/KNN/KDTree搜索2.jpg)

In [15]:
import sys
import time
import numpy as np
from collections import Counter
from heapq import heappush, heappop
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

In [2]:
data = load_breast_cancer()
X, y = data['data'], data['target']
train_X, test_X, train_y, test_y = train_test_split(X, y)
print('train size: %s' % len(train_X))
print('test size: %s' % len(test_X))

train size: 426
test size: 143


In [3]:
def minkowski_distance_p(x, y, p=2):
    '''
    计算LP距离
    '''
    x = np.asarray(x)
    y = np.asarray(y)
    common_datatype = np.promote_types(np.promote_types(x.dtype, y.dtype), 'float64')
    x = x.astype(common_datatype)
    y = y.astype(common_datatype)
    if p == np.inf:
        return np.amax(np.abs(y-x), axis=-1)
    elif p == 1:
        return np.sum(np.abs(y-x), axis=-1)
    else:
        return np.sum(np.abs(y-x)**p, axis=-1)

In [4]:
class KDTree(object):
    def __init__(self, data, leafsize=2):
        """
        data: 每个数据点的列表或numpy矩阵
        leafsize: 叶子节点的个数, 节点数据数目小于等于该值不再切分
        """
        self.data = np.asarray(data)
        self.n, self.m = np.shape(self.data)
        self.leafsize = int(leafsize)
        if self.leafsize < 1:
            raise ValueError("leafsize must be at least 1")
        # 返回每一维的最大最小值
        self.maxes = np.amax(self.data, axis=0)
        self.mins = np.amin(self.data, axis=0)
        # 调用build建立KDTree
        self.tree = self.__build(np.arange(self.n), self.maxes, self.mins)
    
    class node(object):
        if sys.version_info[0] >= 3:
            def __lt__(self, other):
                return id(self) < id(other)

            def __gt__(self, other):
                return id(self) > id(other)

            def __le__(self, other):
                return id(self) <= id(other)

            def __ge__(self, other):
                return id(self) >= id(other)

            def __eq__(self, other):
                return id(self) == id(other)
    
    # 叶子节点
    class leafnode(node):
        def __init__(self, idx):
            self.idx = idx
            self.children = len(idx)
    
    # 内部节点
    class innernode(node):
        def __init__(self, split_dim, split, less, greater):
            '''
            split_dim: 切分维度
            split: 切分值
            less: 小于切分值的节点
            greater: 大于切分值的节点
            '''
            self.split_dim = split_dim
            self.split = split
            self.less = less
            self.greater = greater
            self.children = less.children + greater.children

    # 建立KDTree
    def __build(self, idx, maxes, mins):
        '''
        idx: 每个数据点的索引
        maxes: 每一维的最大值
        mins: 每一维的最小值
        '''
        
        # 如果数据点的个数小于预设的叶子节点个数，设为叶子节点
        if len(idx) <= self.leafsize:
            return KDTree.leafnode(idx)
        else:
            data = self.data[idx]

            # 查找极差最大的纬度
            d = np.argmax(maxes-mins)
            # 该维度的最大最小值
            maxval = maxes[d]
            minval = mins[d]
            # 判断所有数据点是否全部相等
            if maxval == minval:
                return KDTree.leafnode(idx)
            # 极差最大的维度数据
            data = data[:,d]
            # 切分值为中值
            split = (maxval + minval) / 2
            # 寻找大于切分值和小于切分值的数据索引
            less_idx = np.nonzero(data <= split)[0]
            greater_idx = np.nonzero(data > split)[0]
            # 如果小于等于切分值的索引数为0
            if len(less_idx) == 0:
                split = np.amin(data)
                less_idx = np.nonzero(data <= split)[0]
                greater_idx = np.nonzero(data > split)[0]
            # 如果大于等于切分值的索引数为0
            if len(greater_idx) == 0:
                split = np.amax(data)
                less_idx = np.nonzero(data < split)[0]
                greater_idx = np.nonzero(data >= split)[0]
            # 如果仍然为0，则所有的数据均相等
            if len(less_idx) == 0:
                if not np.all(data == data[0]):
                    raise ValueError("Troublesome data array: %s" % data)
                split = data[0]
                less_idx = np.arange(len(data)-1)
                greater_idx = np.array([len(data)-1])
            # 左孩子节点的切分维度的最大值该为切分值
            lessmaxes = np.copy(maxes)
            lessmaxes[d] = split
            # 右孩子节点的切分维度的最小值该为切分值
            greatermins = np.copy(mins)
            greatermins[d] = split
            # 递归建立树
            return KDTree.innernode(d, split,
                    self.__build(idx[less_idx], lessmaxes, mins),
                    self.__build(idx[greater_idx], maxes, greatermins))

    def __query(self, x, k=1, eps=0, p=2, distance_upper_bound=np.inf):
        '''
        查询单个数据点
        '''
        # 边界距离
        side_distances = np.maximum(0, np.maximum(x - self.maxes, self.mins - x))
        if p != np.inf:
            side_distances **= p
            min_distance = np.sum(side_distances)
        else:
            min_distance = np.amax(side_distances)
        
        q = [(min_distance, tuple(side_distances), self.tree)]
        neighbors = []

        if eps == 0:
            epsfac = 1
        elif p == np.inf:
            epsfac = 1/(1+eps)
        else:
            epsfac = 1/(1+eps)**p

        if p != np.inf and distance_upper_bound != np.inf:
            distance_upper_bound = distance_upper_bound**p

        while q:
            min_distance, side_distances, node = heappop(q)
            # 如果是叶子节点
            if isinstance(node, KDTree.leafnode):
                # brute-force
                # 对叶子节点的数据点暴力搜索
                data = self.data[node.idx]
                ds = minkowski_distance_p(data, x[np.newaxis,:], p)
                for i in range(len(ds)):
                    # 如果小于最大距离则加入堆
                    if ds[i] < distance_upper_bound:
                        if len(neighbors) == k:
                            heappop(neighbors)
                        heappush(neighbors, (-ds[i], node.idx[i]))
                        if len(neighbors) == k:
                            distance_upper_bound = -neighbors[0][0]
            # 如果是内部节点
            else:
                # 如果最近距离都超过上限，停止搜索
                if min_distance > distance_upper_bound*epsfac:
                    break
                # 判断搜索方向
                if x[node.split_dim] < node.split:
                    near, far = node.less, node.greater
                else:
                    near, far = node.greater, node.less
                heappush(q, (min_distance, side_distances, near))
                
                sd = list(side_distances)
                if p == np.inf:
                    min_distance = max(min_distance, abs(node.split - x[node.split_dim]))
                elif p == 1:
                    sd[node.split_dim] = np.abs(node.split - x[node.split_dim])
                    min_distance = min_distance - side_distances[node.split_dim] + sd[node.split_dim]
                else:
                    sd[node.split_dim] = np.abs(node.split-x[node.split_dim])**p
                    min_distance = min_distance - side_distances[node.split_dim] + sd[node.split_dim]

                if min_distance <= distance_upper_bound*epsfac:
                    heappush(q, (min_distance, tuple(sd), far))

        if p == np.inf:
            return sorted([(-d,i) for (d,i) in neighbors])
        else:
            return sorted([((-d)**(1./p),i) for (d,i) in neighbors])

    def query(self, x, k=1, eps=0, p=2, distance_upper_bound=np.inf):
        """
        x : 待查询数据点的numpy矩阵
        k : k近邻点
        eps : 容忍度，查询的第K个点的距离不超过实际第K个点的（1+eps）倍
        p : LP距离
        distance_upper_bound : 距离上限
        Returns
        -------
        d : 最近点的距离
        i : 最近点的索引
        """
        x = np.asarray(x)
        if np.shape(x)[-1] != self.m:
            raise ValueError("x must consist of vectors of length %d but has shape %s" % (self.m, np.shape(x)))
        if p < 1:
            raise ValueError("Only p-norms with 1<=p<=infinity permitted")
        retshape = np.shape(x)[:-1]
        if retshape != ():
            if k is None:
                dd = np.empty(retshape,dtype=object)
                ii = np.empty(retshape,dtype=object)
            elif k > 1:
                dd = np.empty(retshape+(k,),dtype=float)
                dd.fill(np.inf)
                ii = np.empty(retshape+(k,),dtype=int)
                ii.fill(self.n)
            elif k == 1:
                dd = np.empty(retshape,dtype=float)
                dd.fill(np.inf)
                ii = np.empty(retshape,dtype=int)
                ii.fill(self.n)
            else:
                raise ValueError("Requested %s nearest neighbors; acceptable numbers are integers \
                                    greater than or equal to one, or None")
            for c in np.ndindex(retshape):
                hits = self.__query(x[c], k=k, eps=eps, p=p, distance_upper_bound=distance_upper_bound)
                if k is None:
                    dd[c] = [d for (d,i) in hits]
                    ii[c] = [i for (d,i) in hits]
                elif k > 1:
                    for j in range(len(hits)):
                        dd[c+(j,)], ii[c+(j,)] = hits[j]
                elif k == 1:
                    if len(hits) > 0:
                        dd[c], ii[c] = hits[0]
                    else:
                        dd[c] = np.inf
                        ii[c] = self.n
            return dd, ii
        else:
            hits = self.__query(x, k=k, eps=eps, p=p, distance_upper_bound=distance_upper_bound)
            if k is None:
                return [d for (d,i) in hits], [i for (d,i) in hits]
            elif k == 1:
                if len(hits) > 0:
                    return hits[0]
                else:
                    return np.inf, self.n
            elif k > 1:
                dd = np.empty(k,dtype=float)
                dd.fill(np.inf)
                ii = np.empty(k,dtype=int)
                ii.fill(self.n)
                for j in range(len(hits)):
                    dd[j], ii[j] = hits[j]
                return dd, ii
            else:
                raise ValueError("Requested %s nearest neighbors; acceptable numbers are\
                                 integers greater than or equal to one, or None")

In [60]:
class KNN:
    def __init__(self, train_X, train_y, k=5):
        self.train_y = train_y
        self.k = k
        self.tree = KDTree(train_X)
        
    def predict(self, test_X):
        _, idxs = self.tree.query(test_X, k=self.k)
        res = []
        if self.k == 1:
            idxs = np.reshape(idxs, (-1, 1))
        for i in idxs:
            pred = self.train_y[i]
            pred = Counter(pred).most_common()[0][0]
            res.append(pred)
        return res

In [65]:
%time
knn = KNN(train_X, train_y, k=5)
preds = knn.predict(test_X)
acc = sum(test_y == preds) / len(preds)
print('算法准确率: %.6f' %acc)

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 4.29 µs
算法准确率: 0.923077
